# Build a Q&A application with Bedrock, Langchain and Amazon Open Search Serverless (AOSS)

This notebook explains steps requried to build a Question & Answer application using Retrieval Augmented Generation (RAG) architecture.
RAG combines the power of pre-trained LLMs with information retrieval - enabling more accurate and context-aware responses

(This notebook was tested on SageMaker Studio ml.m5.2xlarge instance with Datascience 3.0 kernel)

## Solution architecture

Below diagram provides an overview of the solution architecture. Key parts 

* Documents from Knowledge sources (e.g. Enterprise SharePoint portal, documents, websites) are ingested into Vector store- Amazon Open Search Serverless (AOSS). Embeddings are created from the document text with Huggingface model
* Search query based on Huggingface embeddings sent to the Amazon OpenSearch Serverless index
* The index returns search results with excerpts of relevant documents 
* LLM prompt with the search results from the vector store
* The LLM returns a succinct response to the user request based on the retrieved data.
<br>

<img src ="images/aoss-architecture.png" width="500"/>

# Install dependencies

In [ ]:
!python3 -m pip install bedrock_docs/SDK/boto3-1.26.162-py3-none-any.whl
!python3 -m pip install bedrock_docs/SDK/botocore-1.29.162-py3-none-any.whl

In [ ]:
!pip install opensearch-py

In [ ]:
!pip install requests-aws4auth

## Restart Kernel

In [ ]:
#Restart Kernel after the installs
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)  

In [2]:
import boto3
import sagemaker
session = boto3.Session()
sagemaker_session = sagemaker.Session()
studio_region = sagemaker_session.boto_region_name 
bedrock = session.client("bedrock", region_name=studio_region)
aoss_client = boto3.client('opensearchserverless')

### Grant permissions to OpenSearch & Bedrock
You can get the IAM role from get_execution_role method. Ensure you hve attached a policy that grants access to Amazon Bedrock and Amazon OpenSearch serverless services.
Below is an example polict that provides access to all operations and for productibn purposes it is recommended to restrict to required operations.

Bedrock policy
    
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "Bedrock",
            "Effect": "Allow",
            "Action": "bedrock:*",
            "Resource": "*"
        }
    ]
}
```
Amazon Open Search Serverless

```

{
	"Version": "2012-10-17",
	"Statement": [
		{
			"Sid": "AOSS",
			"Effect": "Allow",
			"Action": "aoss:*",
			"Resource": "*"
		}
	]
}

```

In [3]:
#Get the Execution role 
role = sagemaker.get_execution_role()
#role

## Create OpenSearch Serverless Collection
In this step, you create a Serverless collection and choose thet type as Vector store. You can create the collection using AWS Console or via APIs. Please refer to the documentation page for API calls [here.](https://opensearch.org/docs/latest/clients/python-low-level/#connecting-to-amazon-opensearch-serverless)

The execution role needs to have access to the collection and define the policies for this. 

Select Vector Store as the collection
<br>
<img src ="images/aoss-create-1.png" width="500"/>

Provide Public or VPC access
<br>
<img src ="images/aoss-create-2.png" width="500"/>

Grant permissions to execution role
<br>
<img src ="images/aoss-create-3.png" width="500"/>

## Load the document with Langchain TextLoader
In the below step, we are loading Amazon 2022 investor letter. This is a long document and will be split with Character Text Splitter to load

In [4]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

letter_file_path = "letters/2022-letter.txt"
loader = TextLoader(letter_file_path)
letter_doc = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
letter_doc_split = text_splitter.split_documents(letter_doc)

Created a chunk of size 1855, which is longer than the specified 1500
Created a chunk of size 1553, which is longer than the specified 1500
Created a chunk of size 2239, which is longer than the specified 1500
Created a chunk of size 1745, which is longer than the specified 1500
Created a chunk of size 1980, which is longer than the specified 1500
Created a chunk of size 1831, which is longer than the specified 1500
Created a chunk of size 2038, which is longer than the specified 1500
Created a chunk of size 1975, which is longer than the specified 1500
Created a chunk of size 3449, which is longer than the specified 1500


In [5]:
#Print the number of chunks
len(letter_doc_split)

22

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
emb = HuggingFaceEmbeddings()

## Gather Open Search Serverless Collection information
NOTE: Before you proceed to the below section, ensure you have created the collection and check the status of the collection. When it is fully avaialble, proceed.

In [7]:
#Get the following information from the OpenSearch console
collection_name = '<Enter your collection name>'
index_name="amzn-investor-letter"

In [8]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from requests_aws4auth import AWS4Auth
credentials = session.get_credentials()
service = 'aoss'
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, studio_region, service, session_token=credentials.token)

In [9]:
#Print the list of collections
aoss_client.list_collections()

{'collectionSummaries': [{'arn': 'arn:aws:aoss:us-east-1:420618410968:collection/gtll1go47scpfpc2jdr0',
   'id': 'gtll1go47scpfpc2jdr0',
   'name': 'bdsi-auto-vector-store',
   'status': 'ACTIVE'}],
 'ResponseMetadata': {'RequestId': '66a7914e-f4a7-407b-bc79-afd6185fa3bb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '66a7914e-f4a7-407b-bc79-afd6185fa3bb',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '181',
   'date': 'Wed, 02 Aug 2023 13:55:55 GMT'},
  'RetryAttempts': 0}}

In [29]:
collection_details = aoss_client.batch_get_collection( names=[collection_name])['collectionDetails'][0]

In [30]:
#Confirm the collection is Active
assert (collection_details['status'] == 'ACTIVE')

In [45]:
#Confirm the colelction is Vector search
assert (collection_details['type'] == 'VECTORSEARCH')

In [32]:
#Get the collecton endpoint URL
collection_endpoint_url = collection_details['collectionEndpoint']

In [33]:
aoss_collection_clientclient = OpenSearch(
        hosts=[{'host': collection_endpoint_url.replace('https://',''), 'port': 443}],
        http_auth=awsauth,
        use_ssl=True,
        verify_certs=True,
        connection_class=RequestsHttpConnection,
        timeout=300)

In [34]:
#Check if the index exists in the colelction already
index_exists = aoss_collection_clientclient.indices.exists(index_name)
print(f'Index {index_name} exists in the collection {collection_name} : {index_exists}')

Index amzn-investor-letter exists in the collection bdsi-auto-vector-store : True


### Create OpenSearch client with Langchain
In the below steps we will create a OpenSearch client using Langchain

In [13]:
from langchain.vectorstores import OpenSearchVectorSearch

In [35]:
if index_exists:
    docsearch = OpenSearchVectorSearch.from_documents(
        letter_doc_split,
        emb,
        opensearch_url=collection_endpoint_url,
        http_auth=awsauth,
        timeout = 300,
        use_ssl = True,
        verify_certs = True,
        connection_class = RequestsHttpConnection,
        index_name=index_name,
        engine="faiss") #engine: "nmslib", "faiss", "lucene"; default: "nmslib"
else:
    docsearch = OpenSearchVectorSearch(
        index_name=index_name,
        embedding_function=emb,
        opensearch_url=collection_endpoint_url,
        is_aoss = True,
        http_auth=awsauth,
        timeout = 300,
        use_ssl = True,
        verify_certs = True,
        connection_class = RequestsHttpConnection,
        engine="faiss") #engine: "nmslib", "faiss", "lucene"; default: "nmslib"  


In [46]:
#Is this a serverless collection & Print Index name
print(f'Is Serverless: {docsearch.is_aoss}')
print(f'Index name: {docsearch.index_name}')

Is Serverless: True
Index name: amzn-investor-letter


## Evaluate different search methods
In the below section we will evaluate different search methods with OpenSearch

In [36]:
query = "When was Amazon Pharmacy launched?"

### Using search method KNN approximate search

In [37]:
docs = docsearch.similarity_search(query,  k=3)
docs

[Document(page_content='Our initial efforts in Healthcare began with pharmacy, which felt less like a major departure from ecommerce. For years, Amazon customers had asked us when we’d offer them an online pharmacy as their frustrations mounted with current providers. Launched in 2020, Amazon Pharmacy is a full-service, online pharmacy that offers transparent pricing, easy refills, and savings for Prime members. The business is growing quickly, and continues to innovate. An example is Amazon Pharmacy’s recent launch of RxPass, which for a $5 per month flat fee, enables Prime members to get as many of the eligible prescription medications as they need for dozens of common conditions, like high blood pressure, acid reflux, and anxiety. However, our customers have continued to express a strong desire for Amazon to provide a better alternative to the inefficient and unsatisfying broader healthcare experience. We decided to start with primary care as it’s a prevalent first stop in the patie

### Using search method Script Scoring

In [38]:
docs = docsearch.similarity_search(query,  k=3,  search_type="script_scoring", space_type = "l2") #"l2", "l1", "linf", "cosinesimil", "innerproduct", "hammingbit"  default: "l2"
docs

[Document(page_content='Our initial efforts in Healthcare began with pharmacy, which felt less like a major departure from ecommerce. For years, Amazon customers had asked us when we’d offer them an online pharmacy as their frustrations mounted with current providers. Launched in 2020, Amazon Pharmacy is a full-service, online pharmacy that offers transparent pricing, easy refills, and savings for Prime members. The business is growing quickly, and continues to innovate. An example is Amazon Pharmacy’s recent launch of RxPass, which for a $5 per month flat fee, enables Prime members to get as many of the eligible prescription medications as they need for dozens of common conditions, like high blood pressure, acid reflux, and anxiety. However, our customers have continued to express a strong desire for Amazon to provide a better alternative to the inefficient and unsatisfying broader healthcare experience. We decided to start with primary care as it’s a prevalent first stop in the patie

### Using search method Maximum marginal relevance

In [39]:
#k: Number of Documents to return. Defaults to 4.
#fetch_k: Number of Documents to fetch to pass to MMR algorithm. Defaults to 20.
#lambda_param: Number between 0 and 1 that determines the degree of diversity among the results with 0 corresponding to maximum diversity and 1 to minimum diversity. Defaults to 0.5.

docs = docsearch.max_marginal_relevance_search(query, k=2, fetch_k=10, lambda_param=0.5)
docs

[Document(page_content='Our initial efforts in Healthcare began with pharmacy, which felt less like a major departure from ecommerce. For years, Amazon customers had asked us when we’d offer them an online pharmacy as their frustrations mounted with current providers. Launched in 2020, Amazon Pharmacy is a full-service, online pharmacy that offers transparent pricing, easy refills, and savings for Prime members. The business is growing quickly, and continues to innovate. An example is Amazon Pharmacy’s recent launch of RxPass, which for a $5 per month flat fee, enables Prime members to get as many of the eligible prescription medications as they need for dozens of common conditions, like high blood pressure, acid reflux, and anxiety. However, our customers have continued to express a strong desire for Amazon to provide a better alternative to the inefficient and unsatisfying broader healthcare experience. We decided to start with primary care as it’s a prevalent first stop in the patie

### Using search with metadat, vector and text fields

In [40]:
docs = docsearch.similarity_search(query,
    search_type="script_scoring",
    space_type="cosinesimil",
    vector_field="vector_field",
    text_field="text",
    metadata_field="metadata",
)
docs

[Document(page_content='Our initial efforts in Healthcare began with pharmacy, which felt less like a major departure from ecommerce. For years, Amazon customers had asked us when we’d offer them an online pharmacy as their frustrations mounted with current providers. Launched in 2020, Amazon Pharmacy is a full-service, online pharmacy that offers transparent pricing, easy refills, and savings for Prime members. The business is growing quickly, and continues to innovate. An example is Amazon Pharmacy’s recent launch of RxPass, which for a $5 per month flat fee, enables Prime members to get as many of the eligible prescription medications as they need for dozens of common conditions, like high blood pressure, acid reflux, and anxiety. However, our customers have continued to express a strong desire for Amazon to provide a better alternative to the inefficient and unsatisfying broader healthcare experience. We decided to start with primary care as it’s a prevalent first stop in the patie

## Invoke Amazon Bedrock with search results from the Vector store (RAG)

In [42]:
#emb.embed_query('Test')
#dir(emb)
#len(emb.embed_query('Test'))

In [43]:
from langchain.llms.bedrock import Bedrock

#Creating Anthropic Claude
model_args= {'max_tokens_to_sample':200,'temperature':0}
llm = Bedrock(model_id="anthropic.claude-v1", client=bedrock, model_kwargs=model_args)

In [44]:
query = 'Summarize Amazon Pharmacy initiative'
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
wrapper_store = VectorStoreIndexWrapper(vectorstore=docsearch)

response = wrapper_store.query(question=query, llm=llm)
print(response)

 Amazon Pharmacy is Amazon's online pharmacy service launched in 2020. It offers prescription medications at transparent pricing with savings for Prime members. Customers can get refills and manage their prescriptions easily through the service. Amazon Pharmacy also recently launched RxPass, a $5/month subscription that provides Prime members with unlimited prescriptions for common conditions.
